List of original JSON attributes:
* msgID
* msgVer
* gwID
* dateTime
* Regulator.Voltage
* Regulator.Current
* Regulator.Power
* Regulator.Battery_voltage
* Regulator.Battery charging current
* Regulator.Battery charging power
* Regulator.Load voltage
* Regulator.Load current
* Regulator.Load power
* Regulator.Battery Temperature
* Regulator.Temperature inside case
* Regulator.Power component temperature
* Regulator.Batterys remaining capacity
* Regulator.Remote battery temperature
* Regulator.Batterys rated power
* Regulator.Battery status
* Regulator.Charing equipment status
* Regulator.Maximum input volt(PV) today
* Regulator.Minimum input volt(PV) today
* Regulator.Maximal battery volt today
* Regulator.Minimum battery volt today
* Regulator.Consumed energy today
* Regulator.Consumed energy this month
* Regulator.Consumed energy this year
* Regulator.Total consumed energy
* Regulator.Generated energy today
* Regulator.Generated energy this month
* Regulator.Generated energy this year
* Regulator.Total generated energy
* Regulator.Carbon dioxide reduction
* Regulator.Battery current
* Temp.Ambient Temp
* Temp.Battery Temp
* Light.LUX
* Temp/Humi.Temperature in lab
* Temp/Humi.Humidity in lab
* Rated value.Panels rated voltage
* Rated value.Panels rated current
* Rated value.Panels rated power
* Rated value.Batterys voltage
* Rated value.Charging current to battery 
* Rated value.Charging power to battery
* Rated value.Rated output current of load
* WetterOnline.info.locations
* WetterOnline.info.type
* WetterOnline.x0411.data.dd_deg
* WetterOnline.x0411.data.dd_dir
* WetterOnline.x0411.data.ff_bft
* WetterOnline.x0411.data.ff_ms
* WetterOnline.x0411.data.fx_bft
* WetterOnline.x0411.data.fx_kmh
* WetterOnline.x0411.data.pop
* WetterOnline.x0411.data.pp_hpa
* WetterOnline.x0411.data.rad_wm2
* WetterOnline.x0411.data.rh
* WetterOnline.x0411.data.rr_mm
* WetterOnline.x0411.data.tt_C
* WetterOnline.x0411.data.tta_C
* WetterOnline.x0411.data.wm
* WetterOnline.x0411.meta.local_date

In [1]:
import io
import json
import re
import sys

import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

import pandas as pd
from pandas.io.json import json_normalize

from tqdm import tqdm

from azure.storage.blob import BlockBlobService
# See documentation
# https://azure-storage.readthedocs.io/ref/azure.storage.blob.baseblobservice.html
# https://azure-storage.readthedocs.io/ref/azure.storage.blob.models.html
# https://github.com/Microsoft/AzureNotebooks/blob/master/Samples/Access%20your%20data%20in%20Azure%20Notebooks.ipynb

pd.options.display.max_rows = 100

In [2]:
load_data_from_azure = False
azure_storage_account_name = "redischoolstorage"
azure_storage_account_key = "<see Azure portal>"
azure_container = 'iotdataavro'
azure_blob_prefix = 'redischoolhub/03/2019/10'
export_pickle_filename = '2019_10.pickle'
export_csv_filename = '2019_10.csv'

# Load data from Azure Blob Storage

In [3]:
if load_data_from_azure:
    
    # Connect to our storage account.
    blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)

    # List containers.
    containers = blob_service.list_containers()
    # for c in containers:
    #    print(c.name)

    df_total = pd.DataFrame()

    blobs = []
    for blob in blob_service.list_blobs(azure_container, prefix=azure_blob_prefix):

        # Only include data that was collected between 7am and 19pm (daylight, roughly).
        # match = re.match('redischoolhub\/\d{2}\/\d{4}\/\d{2}\/\d{2}\/(\d{2})\/\d{2}', blob.name)
        # hour = int(match.group(1))
        # if hour >= 5 and hour <= 17:
        #     blobs.append(blob.name)
        blobs.append(blob.name)

    with tqdm(total=len(blobs), file=sys.stdout) as pbar:    
        for i, name in enumerate(blobs):
            pbar.update(1)

            blob = blob_service.get_blob_to_bytes(azure_container, name)

            # Open the binary Avro downloaded from Azure.
            reader = DataFileReader(io.BytesIO(blob.content), DatumReader())
            for elem in reader:
                body = elem['Body']

                # The JSON in the Avro is a byte object, not a string.
                json_str = str(body, "utf8")

                # The JSON is not really a JSON because it uses single quotes instead of double quotes.
                json_str = json_str.replace("'", '"')

                # Now we can parse the JSON. It is quite complicated, but here comes Pandas.
                json_df = json_normalize(json.loads(json_str))

                df_total = df_total.append(json_df)
            reader.close()

    df_total.to_pickle(export_pickle_filename)
    df_total

# Clean up data

In [4]:
df_total = pd.read_pickle(export_pickle_filename)
df_total

,msgID,msgVer,gwID,dateTime,Regulator.Voltage,Regulator.Current,Regulator.Power,Regulator.Battery_voltage,Regulator.Battery charging current,Regulator.Battery charging power,...,WetterOnline.x0411.data.fx_kmh,WetterOnline.x0411.data.pop,WetterOnline.x0411.data.pp_hpa,WetterOnline.x0411.data.rad_wm2,WetterOnline.x0411.data.rh,WetterOnline.x0411.data.rr_mm,WetterOnline.x0411.data.tt_C,WetterOnline.x0411.data.tta_C,WetterOnline.x0411.data.wm,WetterOnline.x0411.meta.local_date
0,msg0,1.0,MGate 5105_8352,2019-10-08T12:12:11+00:00,1359,38,533,1241,44,521,...,24,50,1015.9,150,75,0.3,11.1,11.1,bdr1__,2019-10-08 12:07:40
0,msg0,1.0,MGate 5105_8352,2019-10-08T12:13:11+00:00,3460,29,1020,1245,81,996,...,24,50,1015.9,150,75,0.3,11.1,11.1,bdr1__,2019-10-08 12:07:40
0,msg0,1.0,MGate 5105_8352,2019-10-08T12:14:11+00:00,1862,32,621,1242,48,608,...,24,50,1015.9,150,75,0.3,11.1,11.1,bdr1__,2019-10-08 12:07:40
0,msg0,1.0,MGate 5105_8352,2019-10-08T12:15:11+00:00,1360,36,508,1241,39,496,...,24,40,1015.7,170,75,0.0,11.3,11.3,bd____,2019-10-08 12:12:39
0,msg0,1.0,MGate 5105_8352,2019-10-08T12:16:11+00:00,1363,37,521,1241,42,508,...,24,40,1015.7,170,75,0.0,11.3,11.3,bd____,2019-10-08 12:12:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,msg0,1.0,MGate 5105_8352,2019-10-21T18:03:25+00:00,1368,8,101,1266,8,113,...,16,30,1019.7,45,70,0.0,14.8,14.8,wb____,2019-10-21 18:01:28
0,msg0,1.0,MGate 5105_8352,2019-10-21T18:04:25+00:00,1368,8,101,1266,7,113,...,16,30,1019.7,45,70,0.0,14.8,14.8,wb____,2019-10-21 18:02:28
0,msg0,1.0,MGate 5105_8352,2019-10-21T18:05:25+00:00,1367,8,113,1266,8,101,...,16,30,1019.7,45,70,0.0,14.8,14.8,wb____,2019-10-21 18:02:28
0,msg0,1.0,MGate 5105_8352,2019-10-21T18:06:25+00:00,1366,7,113,1266,9,101,...,16,30,1019.7,45,70,0.0,14.8,14.8,wb____,2019-10-21 18:01:28


## Drop rows with null values

In [5]:
# Do we have null values?
null_columns=df_total.columns[df_total.isnull().any()]
df_total[null_columns].isnull().sum()

Series([], dtype: float64)

In [6]:
# Drop rows with null values.
df_total = df_total.dropna(axis=0)

## Add DateTime index for convenient indexing

In [7]:
df_total['dateTime_local'] = df_total['dateTime'].str.replace('\+00','+02')
df_total['dateTime_local'] = pd.to_datetime(df_total['dateTime_local']).dt.tz_convert('Europe/Berlin')
df_total.set_index(['dateTime_local'], inplace=True)

## One-hot encode a few columns

In [8]:
# One-hot encode the WetterOnline.x0411.data.wm attribute.
df_wm_dummies = pd.get_dummies(df_total['WetterOnline.x0411.data.wm'], prefix='WetterOnline.wm')
df_total = pd.concat([df_total, df_wm_dummies], axis=1)

In [9]:
# One-hot encode the WetterOnline.x0411.data.dd_dir attribute.
df_dd_dir_dummies = pd.get_dummies(df_total['WetterOnline.x0411.data.dd_dir'], prefix='WetterOnline.dd_dir')
df_total = pd.concat([df_total, df_dd_dir_dummies], axis=1)

## Compute linear correlation between variables

In [10]:
# Compute correlation between variables.
df_corr = df_total.corr(method='pearson')['Regulator.Power']

In [11]:
# Only keep columns that have a decent linear correlation.
df = df_total[[
    'Regulator.Power',
    'Light.LUX',
    'Regulator.Temperature inside case',
    'Temp/Humi.Humidity in lab',
    'WetterOnline.x0411.data.pop',
    'WetterOnline.x0411.data.rad_wm2',
    'WetterOnline.x0411.data.rh',
    'WetterOnline.x0411.data.tt_C',
    'WetterOnline.x0411.data.tta_C',
    'WetterOnline.wm_so____']]

In [12]:
df.corr(method='pearson')['Regulator.Power']

Regulator.Power                      1.000000
Light.LUX                            0.270440
Regulator.Temperature inside case    0.565791
Temp/Humi.Humidity in lab           -0.280966
WetterOnline.x0411.data.pop         -0.175190
WetterOnline.x0411.data.rad_wm2      0.496507
WetterOnline.x0411.data.rh          -0.460869
WetterOnline.x0411.data.tt_C         0.296531
WetterOnline.x0411.data.tta_C        0.295639
WetterOnline.wm_so____               0.272913
Name: Regulator.Power, dtype: float64

In [13]:
# Export to CSV
df.to_csv(export_csv_filename, index=False)